In [2]:
import numpy as np
import warnings
from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt
from yahoo_fin import stock_info as si
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

import tools
import importlib
importlib.reload(tools)

pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")
yf.pdr_override()

### Experiment

In [3]:
yf.download('CL=F', start='2010-01-01')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-12-31,79.279999,80.000000,79.160004,79.360001,79.360001,120372
2010-01-04,79.629997,81.680000,79.629997,81.510002,81.510002,263542
2010-01-05,81.629997,82.000000,80.949997,81.769997,81.769997,258887
2010-01-06,81.430000,83.519997,80.849998,83.180000,83.180000,370059
2010-01-07,83.199997,83.360001,82.260002,82.660004,82.660004,246632
2010-01-08,82.650002,83.470001,81.800003,82.750000,82.750000,310377
2010-01-11,82.879997,83.949997,81.959999,82.519997,82.519997,296304
2010-01-12,82.070000,82.339996,79.910004,80.790001,80.790001,333866
2010-01-13,80.059998,80.669998,78.370003,79.650002,79.650002,401627


### Download data from AWS S3

In [3]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.download_data_from_s3(s3_bucket, local_path)

In [4]:
num_of_years = 10
start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years) + 10)
end = dt.date.today()

### Sub-routine

In [5]:
def stock_begin_timegap(stock):
    num_of_years = 10
    start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years))
    try:
        df = yf.download(stock, start=start)
        stock_start = df.index[0]
        coeff_var = df['Close'].std() / df['Close'].mean()
        return np.array(((stock_start - pd.to_datetime(start)).days, coeff_var))
    except:
        return np.array((10000, 10000))

### Pre-process US and HK stocks

In [6]:
csv_dict = {
    'us' : 'us_stocks.csv',
    'hk' : 'clean_hk_stock.csv'
}
df_dict = dict()
dataset_dict = dict()

In [33]:
%%time
for key, value in csv_dict.items():
    df = pd.read_csv('../data/us_hk/{}'.format(value), header=None)
    df = df.rename(columns={0: 'stock'})
    #
    df['gap'], df['coeff_var'] = zip(*df['stock'].apply(stock_begin_timegap))
    stock_df = df[(df['gap']<=5) & (df['coeff_var']<1000)]
    stock_df.to_csv('../data/{}_10yr_stock_qualified.csv'.format(key))
    #
    tickers = stock_df['stock'].to_list()
    dataset_dict[key] = pdr.get_data_yahoo(tickers, start, end)['Adj Close']
    dataset_dict[key].to_csv('../data/{}_10yr_stock_close.csv'.format(key))
    #

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ABX: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TSS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 0161.HK: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 0606.HK: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  714 of 714 completed
CPU times: user 3min 38s, sys: 16.8 s, total: 3min 55s
Wall time: 7min 32s


### Use close on all stocks to compile result

In [34]:
dataset_dict['us'].shape

(2615, 98)

In [35]:
dataset_dict['hk'].shape

(2573, 714)

### Remove Null in Dataset

In [36]:
def count_null(dataset):
    null_count = list()
    for i in dataset.index:
        null_dict = {
            'index' : i,
            'count' : dataset.loc[i].isna().sum()
        }
        null_count.append(null_dict)
    null_df = pd.DataFrame(null_count)
    null_df['index'] = pd.to_datetime(null_df['index'])
    null_df.set_index('index', inplace=True)
    return null_df

In [37]:
null_count = dict()
for k, v in dataset_dict.items():
    null_count[k] = count_null(v)

### Remove null in US stocks

In [38]:
udf = null_count['us']
udf[udf['count']>0]

,count
index,
2011-11-25,4
2012-07-03,4
2012-10-29,98
2012-11-23,4
2012-12-24,4
2013-07-03,4
2013-11-29,4
2013-12-24,4
2014-07-03,4


In [39]:
dataset_dict['us'].loc['2019-03-03']

AAPL             NaN
ADBE             NaN
ADSK             NaN
AIG              NaN
AMAT             NaN
AMD              NaN
AMZN             NaN
AUY              NaN
AVGO             NaN
AXP              NaN
AZN              NaN
BA               NaN
BIDU             NaN
C                NaN
CAT              NaN
CL=F             NaN
CMCSA            NaN
CME              NaN
CREE             NaN
CRM              NaN
DDD              NaN
DE               NaN
DIS              NaN
DLB              NaN
DLR              NaN
EA               NaN
EBAY             NaN
ERJ              NaN
F                NaN
FDX              NaN
FFIV             NaN
FORM             NaN
GC=F     1296.300049
GLD              NaN
GLW              NaN
GM               NaN
GOOG             NaN
GPN              NaN
GPS              NaN
GSM              NaN
HSY              NaN
IAG              NaN
IBM              NaN
IMAX             NaN
INTC             NaN
IRBT             NaN
JNJ              NaN
JPM          

### Examine the weekday of the non-trading date that 90% of stocks are not trading

In [41]:
def find_weekday(x):
    return x.weekday()

In [43]:
# Saturday = 5, Sunday = 6
non_trading_index = udf[udf['count']>90].index
non_trading_df = pd.DataFrame({'Date':non_trading_index})
non_trading_df['WeekDay'] = non_trading_df['Date'].apply(find_weekday)
non_trading_df[non_trading_df['WeekDay']<5]

,Date,WeekDay
0,2012-10-29,0
1,2016-01-18,0
2,2017-01-02,0
3,2017-02-20,0
4,2018-12-05,2


### Why does stock market close on these days ?
2012-10-29 - market close due to Hurricane Sandy<br>
2016-01-18 - ?<br>
2017-01-02 - New Year's Day<br>
2017-02-20 - Washington's Birthday<br>
2018-12-05 - George H.W. Bush<br>

In [44]:
dataset_dict['us'].drop(non_trading_index, inplace=True)

In [45]:
dataset_dict['us'].fillna(method='ffill', inplace=True)

In [46]:
dataset_dict['us'].isna().sum().sum()

0

In [47]:
dataset_dict['us'].to_csv('../data/us_hk/us_clean_10yr_stock_close.csv')

### Remove null in HK stocks

In [48]:
hdf = null_count['hk']
hdf[hdf['count']>0]

,count
index,
2010-12-23,709
2010-12-27,712
2010-12-28,12
2010-12-29,16
2010-12-30,32
2010-12-31,709
2011-01-03,24
2011-01-04,14
2011-01-05,15


In [49]:
hdf[hdf['count']<=700].max()

count    83
dtype: int64

In [50]:
hdf[(hdf['count']>83) & (hdf['count']<700)]

,count
index,


In [51]:
non_trading_index = hdf[hdf['count']>700].index

In [52]:
non_trading_index

DatetimeIndex(['2010-12-23', '2010-12-27', '2010-12-31', '2011-02-02',
               '2011-09-29', '2011-12-27', '2012-01-23', '2012-01-24',
               '2012-01-25', '2012-03-19',
               ...
               '2019-12-26', '2020-01-27', '2020-01-28', '2020-04-13',
               '2020-04-30', '2020-05-01', '2020-06-25', '2020-10-01',
               '2020-10-02', '2020-10-13'],
              dtype='datetime64[ns]', name='index', length=108, freq=None)

In [53]:
# Saturday = 5, Sunday = 6
non_trading_df = pd.DataFrame({'Date':non_trading_index})
non_trading_df['WeekDay'] = non_trading_df['Date'].apply(find_weekday)
non_trading_df[non_trading_df['WeekDay']<5]

,Date,WeekDay
0,2010-12-23,3
1,2010-12-27,0
2,2010-12-31,4
3,2011-02-02,2
4,2011-09-29,3
5,2011-12-27,1
6,2012-01-23,0
7,2012-01-24,1
8,2012-01-25,2
9,2012-03-19,0


In [54]:
dataset_dict['hk'].loc['2010-12-23']

0001.HK         NaN
0002.HK         NaN
0003.HK         NaN
0004.HK         NaN
0005.HK         NaN
0006.HK         NaN
0007.HK         NaN
0008.HK         NaN
0010.HK         NaN
0011.HK         NaN
0012.HK         NaN
0014.HK         NaN
0015.HK         NaN
0016.HK         NaN
0017.HK         NaN
0018.HK         NaN
0019.HK         NaN
0020.HK         NaN
0023.HK         NaN
0025.HK         NaN
0026.HK         NaN
0028.HK         NaN
0031.HK         NaN
0034.HK         NaN
0035.HK         NaN
0038.HK         NaN
0040.HK         NaN
0041.HK         NaN
0043.HK         NaN
0045.HK         NaN
0046.HK         NaN
0047.HK         NaN
0050.HK         NaN
0051.HK         NaN
0052.HK         NaN
0053.HK         NaN
0056.HK         NaN
0057.HK         NaN
0062.HK         NaN
0063.HK         NaN
0064.HK         NaN
0066.HK         NaN
0069.HK         NaN
0071.HK         NaN
0075.HK         NaN
0078.HK         NaN
0081.HK         NaN
0083.HK         NaN
0085.HK         NaN
0086.HK         NaN


In [55]:
dataset_dict['hk'].drop(non_trading_index, inplace=True)

In [56]:
dataset_dict['hk'].fillna(method='ffill', inplace=True)
dataset_dict['hk'].fillna(method='bfill', inplace=True)

In [57]:
dataset_dict['hk'].isna().sum().sum()

0

In [58]:
dataset_dict['hk'].to_csv('../data/us_hk/hk_clean_10yr_stock_close.csv')

### Combine US and HK stock

In [59]:
dataset_dict['us'].shape

(2523, 98)

In [60]:
dataset_dict['hk'].shape

(2465, 714)

In [61]:
dataset = dataset_dict['hk'].join(dataset_dict['us'])

In [62]:
dataset.isna().sum().sum()

5978

In [63]:
dataset.fillna(method='ffill', inplace=True)
dataset.isna().sum().sum()

0

In [64]:
dataset.to_csv('../data/us_hk/us_hk_clean_10yr_stock_close.csv')

### Upload to AWS S3

In [65]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.upload_data_to_s3(s3_bucket, local_path)